In [28]:
import sys
import os
sys.path.insert(0,os.path.abspath('..'))
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from dask import compute,multiprocessing,delayed
import pathlib
import shutil
import glob
import pandas as pd
import vtk
from scipy.spatial import distance
import numpy as np
import SimpleITK as sitk
import itk

In [29]:
def add_z_pt_above_below(pt,z_offset,pt_list):
    above = [pt[0],pt[1],pt[2]+z_offset]
    below = [pt[0],pt[1],pt[2]-z_offset]
    pt_list.append(below)
    pt_list.append(above)
    pt_list.append(pt)
    return pt_list

In [30]:
def validate_pt(pt,im):
    if pt[0]<0:
        pt[0] = 0
    if pt[0]>im.GetSize()[0]:
        pt[0] = im.GetSize()[0]
    if pt[1] < 0:
        pt[1] = 0
    if pt[1] > im.GetSize()[1]:
        pt[1] = im.GetSize()[1]
        
    return pt

In [31]:
def get_min_max(pts):
    min_x = 9999
    max_x = 0
    min_y = 9999
    max_y = 0
    
    for pt in pts:
        if pt[0] < min_x:
            min_x = pt[0]
        if pt[0] > max_x:
            max_x = pt[0]
        if pt[1] < min_y:
            min_y = pt[1]
        if pt[1] > max_y:
            max_y = pt[1]
    return min_x,max_x,min_y,max_y

In [32]:
def get_real_foreground_im(bb_pts,masked_im):
    # get actual ROI as foreground
    pt_list=[]
    pt_list = add_z_pt_above_below(bb_pts[0],0.5,pt_list)
    pt_list = add_z_pt_above_below(bb_pts[1],0.5,pt_list)
    pt_list = add_z_pt_above_below(bb_pts[2],0.5,pt_list)
    pt_list = add_z_pt_above_below(bb_pts[3],0.5,pt_list)

    surf=Surface(pts=pt_list).create_delunay_surface_3d(return_hull=True,output_filename=output_path+'bla.vtk')

    np_im = sitk.GetArrayFromImage(masked_im)
    nonzeros = np.where(np_im>0)
    inds_pts = np.array([nonzeros[0],nonzeros[1],np.zeros_like(nonzeros[0])]).transpose()

    l = Landmarks(pts=inds_pts)
    selected = l.get_landmarks_within_given_surface(surf)
    print(selected)
    
    return selected

In [33]:
def get_bounding_plane(cube,z_coord,tx_mat,im,neg_x=False,coronal=False):
    
    intersectoin_plane = Surface(polydata=cube.get_intersection_plane(z_coord))
    
    if intersectoin_plane.surface.GetNumberOfPoints() > 3:
        
        
        if neg_x:
            neg_mat = [-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
            intersectoin_plane.apply_transformation(neg_mat,inverse=True)
        if coronal:
            coronal_tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]
            intersectoin_plane.apply_transformation(coronal_tr_mat,inverse=True)
            #intersectoin_plane.write_surface_mesh(output_path+'before.vtk')
        intersectoin_plane.apply_transformation(tx_mat,inverse=True)
#         intersectoin_plane_cube = Surface(polydata=intersectoin_plane.create_delunay_surface_3d(return_hull=True,\
#                                                              output_filename=output_path+'vS1_plane.vtk'))
        #intersectoin_plane.write_surface_mesh(output_path+'{}_sec_cutting_plane.vtk'.format(sec_num))
        bounds = intersectoin_plane.surface.GetBounds()
        #pt_list = []
        
        #vtkpts = intersectoin_plane.surface.GetPoints()
        #print(selected)
        #XY_RES = 1
         
#         pt1 = [int(vtkpts.GetPoint(0)[0]/XY_RES),int(vtkpts.GetPoint(0)[1]),0]
#         pt2 = [int(vtkpts.GetPoint(1)[0]/XY_RES),int(vtkpts.GetPoint(1)[1]),0]
#         pt3 = [int(vtkpts.GetPoint(2)[0]/XY_RES),int(vtkpts.GetPoint(2)[1]),0]
#         pt4 = [int(vtkpts.GetPoint(3)[0]/XY_RES),int(vtkpts.GetPoint(3)[1]),0]
        
        # get im inds within the BB of ROI
#         if neg_x:
#             pt1 = validate_pt([-int(bounds[0]/XY_RES),int(bounds[2]/XY_RES),0],im)
#             pt2 = validate_pt([-int(bounds[1]/XY_RES),int(bounds[3]/XY_RES),0],im)
#         else:
        pt1 = validate_pt([int(bounds[0]/XY_RES),int(bounds[2]/XY_RES),0],im)
        pt2 = validate_pt([int(bounds[1]/XY_RES),int(bounds[3]/XY_RES),0],im)
        #pt3 = validate_pt(pt3,im)
        #pt4 = validate_pt(pt4,im)
        
        return [pt1,pt2]
    else:
        return None

In [34]:
def mask_im_region_from_bb(np_im,bb_pts):
    #np_im = sitk.GetArrayFromImage(im)
    #np_im[:,:] = 0
    
    min_x,max_x,min_y,max_y = get_min_max(bb_pts)
    print(min_x,max_x,min_y,max_y)
    np_im[bb_pts[0][1]:bb_pts[1][1],bb_pts[0][0]:bb_pts[1][0]] = 255
    masked_im = sitk.GetImageFromArray(np_im)
    #sitk.WriteImage(masked_im,'{}_binary_masked.tif'.format(op_filename))
    
    return np_im

In [35]:
def mark_binary_im(masked_im):
    contour_im = sitk.BinaryContour(masked_im,backgroundValue=0,foregroundValue=255)
    dil = sitk.BinaryDilateImageFilter()
    dil.SetKernelRadius(10)
    dil.SetBackgroundValue(0)
    dil.SetForegroundValue(255)
    contour_dilated = dil.Execute(contour_im)
    marked_im = sitk.Or(im,contour_dilated)
    return marked_im
    

In [36]:
def get_cylinder(radius,edge):
    center = (np.array(edge[0])+np.array(edge[1]))/2
    height = Vectors().get_vec_length(edge)
    cylinderSource = vtk.vtkCylinderSource()
    cylinderSource.SetCenter(center)
    cylinderSource.SetRadius(radius)
    cylinderSource.SetHeight(height)
    cylinderSource.SetResolution(10)
    cylinderSource.Update()

    pts = []
    ref_pts = []
    pts.append(center)
    end_pt = [center[0],cylinderSource.GetOutput().GetBounds()[2],center[2] ]
    pts.append(end_pt)

    ref_pts.append(center)
    end_pt = Vectors().get_unit_vec(center,edge[1])*height/2 + center
    ref_pts.append(end_pt)
    #Landmarks(pts=pts).write_landmarks(output_path_neun_profiles+'pts')
    #Landmarks(pts=ref_pts).write_landmarks(output_path_neun_profiles+'ref_pts')

    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)

    cyl_surf = Surface(polydata=cylinderSource.GetOutput())
    cyl_surf.apply_icp_transform(icp)
    #print(txmat)

    #Surface(polydata=cylinderSource.GetOutput()).write_surface_mesh(output_path_neun_profiles+'_cylinder.vtk')

    return cyl_surf 

In [66]:
animal_name = 'MG49_lhs'
image_name = 'MG49_3Day_lhs'
section_sg_path = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Original_Data/Spatial_Graphs/misc/Section_Graphs/{}.am/'.format(animal_name)

tx_file = section_sg_path +'aligntobarrel.hx'
file_pattern = '*S*_ct.am'
images_path = '/nas1/Data_aman/00_Rabies/{}/00_Images/00_Confocal/ch_01_stacks/'.format(image_name)
image_file_pattern = 'S*_01.tif'
#rabies_landmarks_path = '/nas1/Data_Mythreya/MotorCortexProject/V4/0_Inputs/SpatialGraphs/Section_Graphs/Rabies/{}/rabies/'.format(animal_name)
#neun_landmarks_path = '/nas1/Data_Mythreya/MotorCortexProject/V0/0_Inputs/Landmarks/Manual/Rabies/NeuN/{}/'.format(animal_name)

surface_path_vS1 = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Surfaces/{}_s1_hull.vtk'.format(animal_name)
surface_path_vM1 = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/vM1_Ref_Surfaces/{}_vM1.vtk'.format(animal_name)

output_path = '/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/{}/'.format(animal_name)

XY_RES = 0.868

exp_name = 'MG49_lhs'
sg_path = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Surfaces/{}_barrels_50.am'.format(exp_name)

barrel_radius = 250 # microns

In [67]:
# Create barrel columns : C2,C3,D2,D3
sg = AmiraSpatialGraph(sg_path,barrel_projections_present=True)
for barrel_row in sg.barrels.rows_list:
    for barrel in barrel_row.single_barrels_list:
        #row = sg.get_barrel_row(row_label=row_label)
        #barrel = row.single_barrels_list[barrel_ind+1]
        barrel_name = barrel.barrel_name
        print(barrel_name)
        if len(barrel.pia_projection_barrel_centroid) > 0 and len(barrel.wm_projection_barrel_centroid)>0:
            edge = [barrel.pia_projection_barrel_centroid,barrel.wm_projection_barrel_centroid]
            sg_edge = AmiraSpatialGraph(generic_graph=True)
            sg_edge.graph_data.add_edge(barrel.pia_projection_barrel_centroid,barrel.wm_projection_barrel_centroid)
            sg_edge.write_spatial_graph(output_path+exp_name+'_'+barrel_name+'_central_axis.am')
            column_surf = get_cylinder(barrel_radius,edge)
            print(output_path+exp_name+'_'+barrel_name+'.vtk')
            column_surf.write_surface_mesh(output_path+exp_name+'_'+barrel_name+'.vtk')
            

A1
/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/MG49_lhs/MG49_lhs_A1.vtk
A2
/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/MG49_lhs/MG49_lhs_A2.vtk
A3
/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/MG49_lhs/MG49_lhs_A3.vtk
A4
/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/MG49_lhs/MG49_lhs_A4.vtk
A5
A6
A7
B1
/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/MG49_lhs/MG49_lhs_B1.vtk
B2
/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/MG49_lhs/MG49_lhs_B2.vtk
B3
/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/MG49_lhs/MG49_lhs_B3.vtk
B4
/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/MG49_lhs/MG49_lhs_B4.vtk
B5
B6
B7
C1
/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/MG49_lhs/MG49_lhs_C1.vtk
C2
/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barre

In [68]:
#sg_3d = AmiraSpatialGraph(glob.glob(path + file_pattern)[0],read_header_only=True)
#landmarks3d = Landmarks()
#landmarks3d_neun = Landmarks()
z_cood_offset = 50
barrels_path = '/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_For_Barrels/'+exp_name
vs1_hull = Surface(surface_path_vS1)
#s1_cube = Surface(polydata=s1_hull.create_delunay_surface_3d(make_cube=True,return_hull=True,\
#                                                             output_filename=output_path+'vS1_Cube.vtk'))

vm1_hull = Surface(surface_path_vM1)
#vm1_cube = Surface(polydata=vm1_hull.create_delunay_surface_3d(make_cube=True,return_hull=True,\
#                                                               output_filename=output_path+'vM1_Cube.vtk'))
#write_cube(vm1_cube.surface,output_path+'vM1_Cube.vtk')

for barrel_surf_filename in glob.glob(barrels_path+'/*.vtk'):
    print(os.path.basename(barrel_surf_filename))
    
    #output_barrel_images_path = barrels_path+'/'+barrel_surf_filename[:-4]+'/'
    pathlib.Path(barrel_surf_filename[:-4]).mkdir(exist_ok=True)
    barrel_surf = Surface(filename=barrel_surf_filename)
    #barrel_surf.create_delunay_surface_3d(make_cube=True,output_filename=output_path+os.path.basename(barrel_surf_filename)[:-4]+'_hull.vtk')
    #print(output_path+barrel_surf_filename[:-4]+'_hull.vtk')
    #continue
    for file in sorted(glob.glob(images_path + image_file_pattern)):
        #sg = AmiraSpatialGraph(file,axis_directions=[1,1,1])
        #print(file,tx_file)

        # read tx mat from the file
        with open(tx_file,'r') as f:
            lines = f.readlines()

        sec_num = int(os.path.basename(file)[1:-7])
        print(sec_num)
        #sec_num = 49

        str_to_compare = '"S{:03d}_ct.am" setTransform '.format(sec_num)
        #print(str_to_compare)
        tx_mat = []
        tx_mat_np = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]).reshape(4,4)
        for line in lines:
            if line.startswith(str_to_compare):
                #tx_mat = (line[len(str_to_compare):-1].split(' '))
                for num in (line[len(str_to_compare):-1].split(' ')):
                    tx_mat.append(float(num))
                tx_mat_np = np.array(tx_mat).reshape(4,4)
                break

        if os.path.exists(images_path+'S{}_01.tif'.format(sec_num)):
            im = sitk.ReadImage(images_path+'S{}_01.tif'.format(sec_num))
            npim = sitk.GetArrayFromImage(im)
            npim[:] = 0
            # get s1 and m1 bounding box for this plane
            s1_bb_pts = get_bounding_plane(barrel_surf,(sec_num-1) * z_cood_offset,tx_mat_np,im, neg_x=False,coronal=False)
            #m1_bb_pts = get_bounding_plane(vm1_hull,(sec_num-1) * z_cood_offset,tx_mat,im, neg_x=True,coronal=False)
            print(s1_bb_pts)
            #print(m1_bb_pts)
            #Landmarks(pts=[s1_bb_pts[0]]).write_landmarks(output_path+'{}_lower.landmarksAscii'.format(sec_num))
            #Landmarks(pts=[s1_bb_pts[1]]).write_landmarks(output_path+'{}_upper.landmarksAscii'.format(sec_num))
            if s1_bb_pts is not None :

                npim = mask_im_region_from_bb(npim,s1_bb_pts,)#output_path+'S'+str(sec_num)+'_vS1')
                #pixel_inds = get_real_foreground_im(s1_bb_pts,s1_masked_im)
                #Landmarks(pts=pixel_inds).write_landmarks(output_path+'s1_im_inds')
                #sitk.WriteImage(cropped_im,output_path+'{}_01_cropped.tif'.format(sec_num))

#             if m1_bb_pts is not None :
#                 npim = mask_im_region_from_bb(npim,m1_bb_pts,)#output_path+'S'+str(sec_num)+'_vM1')
#                 #sitk.WriteImage(cropped_im,output_path+'{}_01_cropped.tif'.format(sec_num))

            #else:
            if s1_bb_pts is not None :#or m1_bb_pts is not None:
                #sitk.WriteImage(sitk.GetImageFromArray(npim),output_path+'S{}_binary.tif'.format(sec_num))
                marked_im = mark_binary_im(sitk.GetImageFromArray(npim))
                sitk.WriteImage(marked_im,barrel_surf_filename[:-4]+'/S{}_boundary_marked.tif'.format(sec_num))

                masked_im = sitk.Mask(im,sitk.GetImageFromArray(npim))
                sitk.WriteImage(masked_im,barrel_surf_filename[:-4]+'/S{}_masked.tif'.format(sec_num))

    

MG49_lhs_A2.vtk
100
None
101
None
102
None
10
[[3713, 3904, 0], [4285, 4472, 0]]
3713 4285 3904 4472
11
[[3152, 3892, 0], [3729, 4452, 0]]
3152 3729 3892 4452
12
[[3061, 4657, 0], [3637, 5217, 0]]
3061 3637 4657 5217
13
[[4150, 4579, 0], [4719, 5151, 0]]
4150 4719 4579 5151
14
[[3643, 4353, 0], [4219, 4913, 0]]
3643 4219 4353 4913
15
[[4115, 5367, 0], [4676, 5943, 0]]
4115 4676 5367 5943
16
[[3804, 5371, 0], [4355, 5948, 0]]
3804 4355 5371 5948
17
[[3982, 5960, 0], [4533, 6537, 0]]
3982 4533 5960 6537
18
[[3513, 5908, 0], [4063, 6485, 0]]
3513 4063 5908 6485
19
[[3249, 5830, 0], [3817, 6403, 0]]
3249 3817 5830 6403
1
None
20
[[4480, 5699, 0], [5032, 6276, 0]]
4480 5032 5699 6276
21
[[4388, 6808, 0], [4941, 7384, 0]]
4388 4941 6808 7384
22
[[4240, 5783, 0], [4816, 6343, 0]]
4240 4816 5783 6343
23
[[4404, 6469, 0], [4968, 7044, 0]]
4404 4968 6469 7044
24
[[4561, 6454, 0], [5120, 7030, 0]]
4561 5120 6454 7030
25
[[4239, 6190, 0], [4803, 6765, 0]]
4239 4803 6190 6765
26
[[4619, 6084, 0], [

13
[[4976, 4124, 0], [5542, 4701, 0]]
4976 5542 4124 4701
14
[[4411, 3812, 0], [4987, 4381, 0]]
4411 4987 3812 4381
15
[[4999, 5093, 0], [5572, 5668, 0]]
4999 5572 5093 5668
16
[[4665, 5044, 0], [5230, 5622, 0]]
4665 5230 5044 5622
17
[[4836, 5624, 0], [5399, 6202, 0]]
4836 5399 5624 6202
18
[[4363, 5575, 0], [4926, 6153, 0]]
4363 4926 5575 6153
19
[[4060, 5400, 0], [4624, 5978, 0]]
4060 4624 5400 5978
1
None
20
[[5321, 5360, 0], [5882, 5939, 0]]
5321 5882 5360 5939
21
[[5221, 6462, 0], [5779, 7041, 0]]
5221 5779 6462 7041
22
[[4981, 5265, 0], [5557, 5834, 0]]
4981 5557 5265 5834
23
[[5213, 6074, 0], [5773, 6653, 0]]
5213 5773 6074 6653
24
[[5406, 6186, 0], [5978, 6761, 0]]
5406 5978 6186 6761
25
[[5041, 5801, 0], [5600, 6380, 0]]
5041 5600 5801 6380
26
[[5461, 5834, 0], [6034, 6408, 0]]
5461 6034 5834 6408
27
[[4693, 6740, 0], [5263, 7316, 0]]
4693 5263 6740 7316
28
[[4778, 6379, 0], [5341, 6958, 0]]
4778 5341 6379 6958
29
[[5679, 6522, 0], [6241, 7100, 0]]
5679 6241 6522 7100
2
None


None
88
None
89
None
8
[[1961, 1778, 0], [2536, 2343, 0]]
1961 2536 1778 2343
90
None
91
None
92
None
93
None
94
None
95
None
96
None
97
None
98
None
99
None
9
[[3597, 3010, 0], [4172, 3571, 0]]
3597 4172 3010 3571
MG49_lhs_C5.vtk
100
None
101
None
102
None
10
[[3018, 2412, 0], [3599, 2979, 0]]
3018 3599 2412 2979
11
[[2390, 2437, 0], [2966, 3009, 0]]
2390 2966 2437 3009
12
[[2313, 3202, 0], [2889, 3774, 0]]
2313 2889 3202 3774
13
[[3571, 3066, 0], [4154, 3626, 0]]
3571 4154 3066 3626
14
[[2925, 2899, 0], [3501, 3471, 0]]
2925 3501 2899 3471
15
[[3878, 3775, 0], [4456, 4349, 0]]
3878 4456 3775 4349
16
[[3478, 3804, 0], [4060, 4370, 0]]
3478 4060 3804 4370
17
[[3652, 4401, 0], [4234, 4965, 0]]
3652 4234 4401 4965
18
[[3197, 4353, 0], [3780, 4917, 0]]
3197 3780 4353 4917
19
[[2777, 4323, 0], [3361, 4882, 0]]
2777 3361 4323 4882
1
None
20
[[4176, 4156, 0], [4759, 4718, 0]]
4176 4759 4156 4718
21
[[4085, 5272, 0], [4666, 5832, 0]]
4085 4666 5272 5832
22
[[3640, 4332, 0], [4216, 4904, 0]]
3

None
55
None
56
None
57
None
58
None
59
None
5
[[3245, 2010, 0], [3378, 2164, 0]]
3245 3378 2010 2164
60
None
61
None
62
None
63
None
64
None
65
None
66
None
67
None
68
None
69
None
6
[[3801, 2217, 0], [4309, 2718, 0]]
3801 4309 2217 2718
70
None
71
None
72
None
73
None
74
None
75
None
76
None
77
None
78
None
79
None
7
[[3924, 3253, 0], [4483, 3832, 0]]
3924 4483 3253 3832
80
None
81
None
82
None
83
None
84
None
85
None
86
None
87
None
88
None
89
None
8
[[3336, 1845, 0], [3899, 2428, 0]]
3336 3899 1845 2428
90
None
91
None
92
None
93
None
94
None
95
None
96
None
97
None
98
None
99
None
9
[[4678, 3839, 0], [5249, 4415, 0]]
4678 5249 3839 4415
MG49_lhs_A1_hull.vtk
100
None
101
None
102
None
10
None
11
[[3532, 4117, 0], [4126, 4723, 0]]
3532 4126 4117 4723
12
[[3445, 4877, 0], [4038, 5483, 0]]
3445 4038 4877 5483
13
[[4475, 4812, 0], [5131, 5479, 0]]
4475 5131 4812 5479
14
[[4035, 4564, 0], [4629, 5170, 0]]
4035 4629 4564 5170
15
[[4335, 5618, 0], [5086, 6377, 0]]
4335 5086 5618 6377
16
[

23
None
24
None
25
None
26
None
27
None
28
None
29
None
2
None
30
None
31
None
32
None
33
None
34
None
35
None
36
None
37
None
38
None
39
None
3
None
40
None
41
None
42
None
43
None
44
None
45
None
46
None
47
None
48
None
49
None
4
None
50
None
51
None
52
None
53
None
54
None
55
None
56
None
57
None
58
None
59
None
5
None
60
None
61
None
62
None
63
None
64
None
65
None
66
None
67
None
68
None
69
None
6
None
70
None
71
None
72
None
73
None
74
None
75
None
76
None
77
None
78
None
79
None
7
None
80
None
81
None
82
None
83
None
84
None
85
None
86
None
87
None
88
None
89
None
8
None
90
None
91
None
92
None
93
None
94
None
95
None
96
None
97
None
98
None
99
None
9
None
MG49_lhs_D5_hull.vtk
100
None
101
None
102
None
10
None
11
None
12
None
13
None
14
None
15
None
16
None
17
[[3844, 3963, 0], [4543, 4663, 0]]
3844 4543 3963 4663
18
[[3380, 3908, 0], [4080, 4607, 0]]
3380 4080 3908 4607
19
[[2933, 3871, 0], [3578, 4516, 0]]
2933 3578 3871 4516
1
None
20
[[4339, 3695, 0], [5034, 4390, 0]]
4339 

...

12
[[2703, 3354, 0], [3279, 3906, 0]]
2703 3279 3354 3906
13
[[3937, 3246, 0], [4511, 3811, 0]]
3937 4511 3246 3811
14
[[3295, 3051, 0], [3871, 3603, 0]]
3295 3871 3051 3603
15
[[4191, 4018, 0], [4742, 4596, 0]]
4191 4742 4018 4596
16
[[3794, 4023, 0], [4354, 4599, 0]]
3794 4354 4023 4599
17
[[3960, 4613, 0], [4521, 5189, 0]]
3960 4521 4613 5189
18
[[3496, 4563, 0], [4058, 5139, 0]]
3496 4058 4563 5139
19
[[3084, 4499, 0], [3657, 5065, 0]]
3084 3657 4499 5065
1
None
20
[[4458, 4357, 0], [5021, 4932, 0]]
4458 5021 4357 4932
21
[[4357, 5468, 0], [4920, 6042, 0]]
4357 4920 5468 6042
22
[[3932, 4483, 0], [4508, 5036, 0]]
3932 4508 4483 5036
23
[[4299, 5138, 0], [4869, 5707, 0]]
4299 4869 5138 5707
24
[[4665, 5120, 0], [5218, 5698, 0]]
4665 5218 5120 5698
25
[[4146, 4860, 0], [4717, 5430, 0]]
4146 4717 4860 5430
26
[[4756, 4755, 0], [5306, 5333, 0]]
4756 5306 4755 5333
27
[[3744, 5874, 0], [4320, 6433, 0]]
3744 4320 5874 6433
28
[[3887, 5457, 0], [4460, 6024, 0]]
3887 4460 5457 6024
29
[[48

11
[[4937, 6034, 0], [5549, 6608, 0]]
4937 5549 6034 6608
12
[[4825, 6791, 0], [5437, 7364, 0]]
4825 5437 6791 7364
13
[[5645, 6894, 0], [6265, 7457, 0]]
5645 6265 6894 7457
14
[[5366, 6470, 0], [5977, 7043, 0]]
5366 5977 6470 7043
15
[[5078, 7906, 0], [5695, 8484, 0]]
5078 5695 7906 8484
16
[[4895, 7837, 0], [5516, 8405, 0]]
4895 5516 7837 8405
17
[[5085, 8400, 0], [5706, 8965, 0]]
5085 5706 8400 8965
18
[[4596, 8336, 0], [5217, 8901, 0]]
4596 5217 8336 8901
19
[[4595, 8100, 0], [5216, 8662, 0]]
4595 5216 8100 8662
1
None
20
[[5555, 8090, 0], [6176, 8653, 0]]
5555 6176 8090 8653
21
[[5468, 9175, 0], [6087, 9736, 0]]
5468 6087 9175 9736
22
[[5798, 7833, 0], [6409, 8406, 0]]
5798 6409 7833 8406
23
[[5602, 8737, 0], [6224, 9295, 0]]
5602 6224 8737 9295
24
[[5394, 8856, 0], [6011, 9432, 0]]
5394 6011 8856 9432
25
[[5394, 8438, 0], [6015, 8996, 0]]
5394 6015 8438 8996
26
[[5381, 8469, 0], [5997, 9048, 0]]
5381 5997 8469 9048
27
[[5230, 9306, 0], [5848, 9874, 0]]
5230 5848 9306 9874
28
[[51

12
[[0, 4873, 0], [0, 5430, 0]]
0 0 4873 5430
13
[[0, 3890, 0], [0, 4447, 0]]
0 0 3890 4447
14
[[0, 4562, 0], [0, 5118, 0]]
0 0 4562 5118
15
[[0, 3004, 0], [0, 3580, 0]]
0 0 3004 3580
16
[[0, 3501, 0], [0, 4073, 0]]
0 0 3501 4073
17
[[0, 4186, 0], [0, 4758, 0]]
0 0 4186 4758
18
[[0, 4128, 0], [0, 4699, 0]]
0 0 4128 4699
19
[[0, 5010, 0], [0, 5569, 0]]
0 0 5010 5569
1
[[0, 3867, 0], [0, 3867, 0]]
0 0 3867 3867
20
[[0, 4007, 0], [0, 4578, 0]]
0 0 4007 4578
21
[[0, 5198, 0], [0, 5768, 0]]
0 0 5198 5768
22
[[0, 5962, 0], [0, 6518, 0]]
0 0 5962 6518
23
[[0, 5387, 0], [0, 5950, 0]]
0 0 5387 5950
24
[[0, 4158, 0], [0, 4733, 0]]
0 0 4158 4733
25
[[0, 5084, 0], [0, 5647, 0]]
0 0 5084 5647
26
[[0, 3657, 0], [0, 4232, 0]]
0 0 3657 4232
27
[[0, 6747, 0], [0, 7298, 0]]
0 0 6747 7298
28
[[0, 5870, 0], [0, 6430, 0]]
0 0 5870 6430
29
[[0, 5929, 0], [0, 6491, 0]]
0 0 5929 6491
2
[[0, 0, 0], [0, 0, 0]]
0 0 0 0
30
[[0, 4606, 0], [0, 5180, 0]]
0 0 4606 5180
31
[[0, 6939, 0], [0, 7495, 0]]
0 0 6939 7495
32

50
None
51
None
52
None
53
None
54
None
55
None
56
None
57
None
58
None
59
None
5
[[0, 134, 0], [0, 707, 0]]
0 0 134 707
60
None
61
None
62
None
63
None
64
None
65
None
66
None
67
None
68
None
69
None
6
[[0, 1367, 0], [0, 1943, 0]]
0 0 1367 1943
70
None
71
None
72
None
73
None
74
None
75
None
76
None
77
None
78
None
79
None
7
[[0, 1038, 0], [0, 1605, 0]]
0 0 1038 1605
80
None
81
None
82
None
83
None
84
None
85
None
86
None
87
None
88
None
89
None
8
[[0, 6181, 0], [0, 6758, 0]]
0 0 6181 6758
90
None
91
None
92
None
93
None
94
None
95
None
96
None
97
None
98
None
99
None
9
[[0, 2540, 0], [0, 3115, 0]]
0 0 2540 3115
MG49_rhs_E1.vtk
100
None
101
None
102
None
10
[[0, 3443, 0], [0, 4020, 0]]
0 0 3443 4020
11
[[0, 3822, 0], [0, 4399, 0]]
0 0 3822 4399
12
[[0, 4586, 0], [0, 5163, 0]]
0 0 4586 5163
13
[[0, 3787, 0], [0, 4362, 0]]
0 0 3787 4362
14
[[0, 4281, 0], [0, 4858, 0]]
0 0 4281 4858
15
[[0, 3252, 0], [0, 3804, 0]]
0 0 3252 3804
16
[[0, 3642, 0], [0, 4202, 0]]
0 0 3642 4202
17
[[0, 4305, 

SystemError: <built-in function BinaryDilateImageFilter_Execute> returned a result with an error set

In [24]:
p = pathlib.Path()
p.mkdir

In [3]:
nr_yr = 5
init = [30000,25000,50000,50000,100000,100000,150000,150000,200000,200000,\
        300000,300000,350000,350000,40000,0,0,0,0,0,\
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
final = 0
interest = 1.05
for i in range(5):
    saving = init[i]
    final = (final +  saving) * interest
final*85

24368000.015625004

In [ ]:
cutPlane = vtk.vtkPlane()
center = Surface(surface_path_vS1).surface.GetCenter()
cutPlane.SetOrigin([center[0],center[1],30])
cutPlane.SetNormal(0, 0, 1)

cutter = vtk.vtkCutter()
cutter.SetInputData(Surface(surface_path_vS1).surface)
cutter.SetCutFunction(cutPlane)
cutter.SetValue(0,0)
cutter.Update()

stripper = vtk.vtkStripper()
stripper.SetInputData(cutter.GetOutput())
stripper.Update()
Surface(polydata=stripper.GetOutput()).write_surface_mesh(output_path+'plane.vtk')
circle = cutter.GetOutput()

In [ ]:
whiteImage =vtk.vtkImageData()
bounds = circle.GetBounds()
whiteImage.SetSpacing([1,1,1])
dim = [int(bounds[1] - bounds[0]),int(bounds[1] - bounds[0]),1 ]
whiteImage.SetDimensions(dim)
whiteImage.SetExtent(0, dim[0] - 1, 0, dim[1] - 1, 0, dim[2] - 1)
whiteImage.SetOrigin([bounds[0],bounds[1],0])
whiteImage.AllocateScalars(vtk.VTK_UNSIGNED_CHAR,1)
inval = 255
outval = 0
count = whiteImage.GetNumberOfPoints()
for i in range(count):
    whiteImage.GetPointData().GetScalars().SetTuple1(i, inval)

extruder =vtk.vtkLinearExtrusionFilter()
extruder.SetInputData(circle)
extruder.SetScaleFactor(1.)
extruder.SetExtrusionTypeToNormalExtrusion()
extruder.SetVector(1, 1, 0)
extruder.Update()
Surface(polydata = extruder.GetOutput()).write_surface_mesh(output_path+'extrusion.vtk')

In [ ]:
pol2stenc =vtk.vtkPolyDataToImageStencil()
pol2stenc.SetTolerance(0) 
pol2stenc.SetInputConnection(extruder.GetOutputPort())
pol2stenc.SetOutputOrigin([bounds[0],bounds[1],0])
pol2stenc.SetOutputSpacing([1,1,1])
pol2stenc.SetOutputWholeExtent(whiteImage.GetExtent())
pol2stenc.Update()

imgstenc = vtk.vtkImageStencil()
imgstenc.SetInputData(whiteImage)
imgstenc.SetStencilConnection(pol2stenc.GetOutputPort())
imgstenc.ReverseStencilOff()
imgstenc.SetBackgroundValue(outval)
imgstenc.Update()

imageWriter = vtk.vtkMetaImageWriter()
imageWriter.SetFileName("labelImage.mhd")
imageWriter.SetInputConnection(imgstenc.GetOutputPort())
imageWriter.Write()

In [ ]:
Surface(polydata=surf).write_surface_mesh(output_path+'plane.vtk')

In [ ]:
surf.get_intersection_plane(30)


In [ ]:
Landmarks(pts=s1_bb_pts).write_landmarks(output_path+'s1_bb_pts')

In [ ]:
intersectoin_plane = Surface(polydata=vm1_cube.get_intersection_plane((sec_num-1) * z_cood_offset))
intersectoin_plane.surface.GetPoints().GetPoint(0)
pt1 = [int(GetPoint(0)[0]/XY_RES),int(GetPoint(0)[1])]
pt2 = [int(GetPoint(1)[0]/XY_RES),int(GetPoint(1)[1])]
pt3 = [int(GetPoint(2)[0]/XY_RES),int(GetPoint(2)[1])]
pt4 = [int(GetPoint(3)[0]/XY_RES),int(GetPoint(3)[1])]


# boundary check and clip if exceeding image boudaries
if pt1 < 0:
    lower[0] = 0
if lower[1] < 0:
    lower[1] = 0
if upper[0] > im.GetWidth():
    upper[0] = im.GetWidth()
if upper[1] > im.GetHeight():
    upper[1] = im.GetHeight()

In [ ]:
5000/XY_RES,1200/XY_RES

In [ ]:
im = sitk.ReadImage(images_path+'S{}_01.tif'.format(sec_num))
transform = sitk.Transform()
transform.SetParameters(tx_mat)
txed_im = sitk.Resample(im,  transform, sitk.sitkLinear, 0.0, sitk.sitkUInt8)
sitk.WriteImage(txed_im,output_path+'S{}.tif'.format(sec_num))

In [ ]:
transform = sitk.Euler2DTransform()
transform.SetTranslation([2592.38,532.041])
transform.SetParameters([tx_mat[0],tx_mat[4],tx_mat[1],tx_mat[5]])

In [ ]:
im = sitk.ReadImage(images_path+'S{}_01.tif'.format(sec_num))
fil = sitk.ResampleImageFilter()
fil.SetInterpolator(sitk.sitkLinear)
fil.SetTransform(transform)
fil.SetDefaultPixelValue(0)
fil.SetOutputPixelType(sitk.sitkUInt8)
fil.SetSize([im.GetWidth()+2000,im.GetHeight()+2000])
#fil.SetOutputOrigin([-2592.38,-532.041,0])
#fil.SetOutputDirection()
txed_im = fil.Execute(im)
sitk.WriteImage(txed_im,output_path+'S{}.tif'.format(sec_num))

In [ ]:
tx_mat

In [ ]:
tx_mat[0],tx_mat[1],tx_mat[4],tx_mat[5]

In [ ]:
animal_name = 

In [30]:
# read the 50um barrels and create a convex hull 
for exp in ['MG49_lhs','MG49_rhs','MG50_lhs','MG50_rhs']:
    barrels_path = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Surfaces/{}_barrels_50.am'.format(exp)
    sg = AmiraSpatialGraph(barrels_path,barrel_projections_present=True)
    s = Surface(pts=sg.barrels.all_rows_graphdata.edge_pt_coords)
    s.create_delunay_surface_3d(return_hull=True)
    s.write_surface_mesh('/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Surfaces/{}_s1_hull_50.vtk'.format(exp))

(vtkCommonDataModelPython.vtkPolyData)0x7f70bf0b2a68